<a href="https://colab.research.google.com/github/Abhijith9999999/Gen-AI-Agentic-Approach/blob/main/AGENTIC_APPROACH_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install 'crewai[tools]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 973.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.3/209.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os


with open('/content/drive/MyDrive/LLM/.bashrc') as file:
    for line in file:
        if line.startswith('export '):
            var, value = line[len('export '):].strip().split('=')
            os.environ[var] = value

# Verify that the environment variable is set

In [ ]:
import json
import os
import re
from datetime import datetime

import requests
from crewai import Agent, Crew, Process, Task
from google.colab import userdata
from IPython.display import Markdown
from langchain.agents import load_tools
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI


def format_response(response: str) -> str:
    entries = re.split(r"(?<=]), (?=\[)", response)
    return [entry.strip("[]") for entry in entries]


os.environ["OPENAI_API_KEY"] = userdata.get("openai-key")

In [ ]:
from crewai_tools import PDFSearchTool
from google.colab import userdata
userdata.get('openai-key')
tool1 = PDFSearchTool(pdf='/content/Businessnews.pdf')
tool2 = PDFSearchTool(pdf='/content/SiriKrishna.pdf')
tool3= PDFSearchTool(pdf='/content/The-Ramayana.pdf')

Inserting batches in chromadb: 100%|██████████| 6/6 [00:04<00:00,  1.36it/s]


In [ ]:
llm = ChatOpenAI(
    #model="gpt-4-turbo-2024-04-09"
    model="gpt-3.5-turbo"
)

In [ ]:
from crewai import Agent, Task, Crew
from fastapi import HTTPException
import numpy as np
import requests
from crewai_tools import PDFSearchTool

# Define the agents with their specific roles and tasks

Businessnews_agent = Agent(
    role=" you are Businessnews analyst",
    goal="Read and provide the similar content from a Business news PDF",
    backstory="""You are an expert in analyzing Business news. You have years of experience working with financial reports, market analyses, and business articles. Your task is to thoroughly read the Businessnews PDF, which contains Business news articles, and provide content that is similar to any given query. If the content does not pertain to Business news, you should acknowledge it as out-of-context. Your expertise will help ensure the most relevant information is provided.""",
    verbose=True,

    allow_delegation=False,
    max_iter=5,
    memory=True,
    tools=[PDFSearchTool(pdf='Businessnews.pdf')]  # Example tool
)

SiriKrishna_agent = Agent(
    role="You are a Scholar in SiriKrisahna's Biography",
    goal="Read and provide similar content from the Lord SiriKrishna PDF",
    backstory="""You are a scholar specializing in the teachings and stories of Lord SiriKrishna. With extensive knowledge in religious texts and interpretations, your task is to read the SiriKrishna PDF, which is focused on Lord SiriKrishna, and provide content that is similar to any given query. If the content does not pertain to Lord SiriKrishna, you should acknowledge it as out-of-context. Your scholarly background ensures that you can provide accurate and insightful information.""",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
    tools=[PDFSearchTool(pdf='SiriKrishna.pdf')]  # Example tool
)

The_Ramayana_agent = Agent(
    role="You are a Scholar in Ramayana ",
    goal="Expertly process and embed content from The Ramayana PDF",
    backstory="""You are an expert in ancient Indian epics, specifically The Ramayana. With deep understanding and years of study in ancient texts, your task is to read the The-Ramayana PDF, which contains content related to The Ramayana, and provide content that is similar to any given query. If the content does not pertain to The Ramayana, you should acknowledge it as out-of-context. Your expertise helps in delivering the most relevant and accurate information from the epic.""",

    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
    tools=[PDFSearchTool(pdf='The-Ramayana.pdf')]  # Example tool
)

query_agent = Agent(
    role="Query Handling Agent",
     goal="Efficiently handle user queries and identify the most relevant document",
    backstory="""As a proficient query handling agent, your role is to receive user queries,find the most relevant content,and also answer the query {query},By carefully analyzing the content of the documents, you ensure that the answers provided are accurate and comprehensive from the relevant document only. Once you identify the relevant document, you collaborate with the appropriate agent to deliver the precise information needed. """,
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
    tools=[]  # No external tools needed for this example
)

# Define the tasks for each agent
task_1 = Task(
    description="Read and provide relevent content from the Business news PDF.",
    expected_output="Content similar to the query from the Business news PDF.",
    agent=Businessnews_agent
)

task_2 = Task(
    description="Read and provide relevant content from the Lord SiriKrishna PDF",
    expected_output="Content similar to the query from the Lord SiriKrishna PDF..",
    agent=SiriKrishna_agent
)

task_3 = Task(
    description="Read and provide relevant content from The Ramayana PDF",
    expected_output="Content similar to the query from The Ramayana PDF.",
    agent=The_Ramayana_agent
)

task_query = Task(
    description="Receive a user query,{query} search which PDFs are related to this topic we have three agents each agents are Businessnews_agent in this it analyzes the content related to the business news. the second agent is SiriKrishna_agent in this the agent gives the topic related to lord SiriKrishna's Biography.and the third agent is The_Ramayana_agent in this it gives the context related to the Ramayana Find which agent is suitable to answer the query from the related topic {query} and give the answer from the relvant content only other wise give out of context question",
    expected_output="Answer to the {query}",
    agent=query_agent,
)

# Assemble the crew
crew = Crew(
    agents=[Businessnews_agent, SiriKrishna_agent, The_Ramayana_agent, query_agent],
    tasks=[ task_query],
    verbose=True,  # Set verbose to a boolean value
    full_output=True,  # Enable full output
    max_iter=15, # Set the maximum number of iterations
)
# Execute the tasks to generate embeddings for PDFs
result = crew.kickoff(inputs={'query': "Who is Sita?"})



 [2024-07-25 01:04:49][DEBUG]: == Working Agent: Query Handling Agent
 [2024-07-25 01:04:49][INFO]: == Starting Task: Receive a user query,Who is Sita? search which PDFs are related to this topic we have three agents each agents are Businessnews_agent in this it analyzes the content related to the business news. the second agent is SiriKrishna_agent in this the agent gives the topic related to lord SiriKrishna's Biography.and the third agent is The_Ramayana_agent in this it gives the context related to the Ramayana Find which agent is suitable to answer the query from the related topic Who is Sita? and give the answer from the relvant content only other wise give out of context question


> Entering new CrewAgentExecutor chain...
Thought: The query "Who is Sita?" is related to the epic Ramayana. Therefore, the relevant agent to consult would be The_Ramayana_agent. I will now retrieve the relevant content from The_Ramayana_agent to provide the most accurate and comprehensive answer.

Fi